##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：函式呼叫設定

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/Function_calling_config.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


指定`function_calling_config`可讓你控制 API 在指定`tools`時的操作方式。例如，你可以選擇只允許純文字輸出 (停用函式呼叫)、強制選擇`tool`中提供的函式子集，或讓它自動執行。

本指南假設你已熟悉函式呼叫。如欲了解簡介，請參考[文件](https://ai.google.dev/docs/function_calling)。


In [ ]:
!pip install -qU 'google-generativeai>=0.5'

要執行下列Cell，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 秘鑰中。如果你還沒有 API 金鑰，或者不確定如何建立 Colab 秘鑰，請參閱 [認證](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門以取得範例。


In [ ]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## 使用工具建立模型

本範例使用 3 個控制器一個簡單假設照明系統的功能。使用這些功能需要按照特定順序呼叫它們。例如，你必須先打開照明系統，然後才能變更顏色。

雖然你可以直接將這些傳遞到模型，並讓它嘗試正確呼叫，但指定 `function_calling_config` 可讓你精確控制模型可使用的功能。


In [ ]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro-latest", tools=light_controls, system_instruction=instruction
)

chat = model.start_chat()

建立一個輔助函式在 `tool_config` 上設定 `function_calling_config`。


In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

## 僅文字模式：`NONE`

如果你提供了工具給模型，但不想在當前對話輪次中使用這些工具，則指定模式為 `NONE`。`NONE` 告訴模型不要呼叫任何函式，並且會表現得好像沒有提供任何函式。


In [ ]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

Hello! As a lighting system, I can turn lights on and off, and I can change the color of the lights. What would you like me to do today? 



## 自動模式：`AUTO`

即使允許模型決定要以文字回應或呼叫特定函式，你也可以將模式指定為 `AUTO`。


In [ ]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])
chat.rewind();  # We're not actually calling the function, so remove this from the history.

function_call {
  name: "enable_lights"
  args {
  }
}



## 函式呼叫模式：`ANY`

設定模式為 `ANY` 將強制模型進行函式呼叫。透過設定 `allowed_function_names`，模型將僅從這些函式中進行選擇。如果沒有設定，則 `tools` 中的所有函式都是函式呼叫的候選函式。

在這個範例系統中，如果燈已經開啟，則使用者可以變更顏色或關閉燈。


In [ ]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "#800080"
      }
    }
  }
}



## 自動函式呼叫

同時在啟用自動函式呼叫時，`tool_config` 會起作用。


In [ ]:
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
auto_chat.send_message("It's awful dark in here...", tool_config=tool_config);

LIGHTBOT: Lights enabled.


## 進一步閱讀

查看函式呼叫[快速入門](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb)，了解函式呼叫簡介。你可以 [這裡](https://github.com/google-gemini/cookbook/blob/main/quickstarts/rest/Function_calling_REST.ipynb) 找到另一個有趣的函式呼叫範例，使用 curl。
